In [1]:
import pandas as pd
from pymongo import MongoClient
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from pandas.io.json import json_normalize

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [3]:
db = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))
df = pd.DataFrame(json_normalize(db))
print(df.shape)
df.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## Now we Drop the rows of all companies except those that are video game companies
We assume that our company will that other video game companies have similar needs to ours. This is before we specify our exact needs.

In [4]:
df1 = df
df1.drop(df1[df1.category_code != "games_video"].index, axis=0, inplace=True)

In [5]:
df1.shape

(956, 12)

In [6]:
df1.head()

,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
5,52cdef7c4bab8bd675297d92,Flektor,games_video,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
11,52cdef7c4bab8bd675297d99,Lala,games_video,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
18,52cdef7c4bab8bd675297da0,Babelgum,games_video,,,,,London,None,GBR,53.344104,-6.267494
19,52cdef7c4bab8bd675297d9d,Joost,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
24,52cdef7c4bab8bd675297daa,Sparter,games_video,None,None,None,None,None,None,USA,37.090240,-95.712891


#### We don't need the "_id" column so let's drop it

In [7]:
df1 = df1.drop(['_id'], axis=1)

In [8]:
df1.head()

,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
5,Flektor,games_video,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
11,Lala,games_video,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
18,Babelgum,games_video,,,,,London,None,GBR,53.344104,-6.267494
19,Joost,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
24,Sparter,games_video,None,None,None,None,None,None,USA,37.090240,-95.712891


In [9]:
df1.dropna(subset = ['offices.city'])
df1.shape
df1["offices.city"].value_counts()

New York         75
San Francisco    68
                 54
London           36
Los Angeles      31
                 ..
Acton             1
Doylestown        1
Santa Ana         1
Budapest          1
Hong Kong         1
Name: offices.city, Length: 363, dtype: int64

#### Have to change name of columns to take out the "."s from the columns due to problems exporting JSON

In [10]:
df1.columns = df1.columns.str.replace('.', '_')

In [11]:
df1.head()

,name,category_code,offices_description,offices_address1,offices_address2,offices_zip_code,offices_city,offices_state_code,offices_country_code,offices_latitude,offices_longitude
5,Flektor,games_video,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
11,Lala,games_video,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
18,Babelgum,games_video,,,,,London,None,GBR,53.344104,-6.267494
19,Joost,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
24,Sparter,games_video,None,None,None,None,None,None,USA,37.090240,-95.712891


 #### We see that most video game companies are in either New York or San Francisco. We export the clean data set as a json to further explore in mongol.

In [12]:
df1.to_json("../gaming_companies_final.json",orient="records")

#### Now we find design companies within the companies database and do the same process as above. This is to satisfy the following requisite 

"Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design."

In [13]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


In [14]:
db = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))
df = pd.DataFrame(json_normalize(db))
print(df.shape)
df.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


In [15]:
df.columns = df.columns.str.replace('.', '_')

In [16]:
df.head(7)

,_id,name,category_code,offices_description,offices_address1,offices_address2,offices_zip_code,offices_city,offices_state_code,offices_country_code,offices_latitude,offices_longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
5,52cdef7c4bab8bd675297d92,Flektor,games_video,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768
6,52cdef7c4bab8bd675297d90,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


## Now we Drop the rows of all companies except those that are design companies.
This is because we want a database with only design companies to specify our first need. We also drop the "_id" column as before

In [17]:
df2 = df
df2.drop(df2[df2.category_code != "design"].index, axis=0, inplace=True)
df2 = df2.drop(['_id'], axis=1)

In [18]:
df2.head()
df2.shape

(6, 11)

In [19]:
df2.offices_city.value_counts()

London           1
San Francisco    1
Berlin           1
Ellensburg       1
Brooklyn         1
Collingwood      1
Name: offices_city, dtype: int64

In [20]:
df2.to_json("../design_companies_final.json",orient="records")

#### From this we see that there is only one design company in Brooklyn (New York) and one in San Francisco. This narrows down our search to two possible points: Brooklyn and San Francisco since those our previous search showed most video game companies being either in San Francisco and New York. 

Since we know our maintenance guy loves basketball, we assume that he prefers to be in San Francisco since they have a better basketball team than Brooklyn or New York, i.e: The warriors are a much better team than the Nets or Knicks